# TM10007 Assignment template

In [86]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [87]:
# Data loading functions. Uncomment the one you want to use
# Import other classifiers you plan to use
from worclipo.load_data import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer


data = load_data()
print(f"The number of samples: {len(data.index)}")
print(f"The number of columns: {len(data.columns)}")
print(f"The number of liposarcoma in the dataset: {len(data[data['label'] == 'liposarcoma'])}")
print(f"The number of lipoma in the dataset: {len(data[data['label'] == 'lipoma'])}")


The number of samples: 115
The number of columns: 494
The number of liposarcoma in the dataset: 58
The number of lipoma in the dataset: 57


In [105]:
def find_zeros_in_data(data):
    # Maak een kopie van de data om de originele data niet te wijzigen
    data_copy = data.copy()
    
    # Vervang waarden die gelijk zijn aan 0 met NaN om ze gemakkelijk te tellen
    data_copy = data_copy.replace(0, np.nan)
    
    # Tel het aantal NaNs in elke kolom, wat overeenkomt met het originele aantal 0's
    zeros_count = data_copy.isna().sum()
    
    # Print het aantal 0's gevonden in elke kolom
    print("Aantal 0's per feature:\n", zeros_count)
    
    # Return het aantal 0's per feature voor verdere analyse indien nodig
    return zeros_count

# Roep de functie aan met je dataset 'data'
zeros_count = find_zeros_in_data(data)

# # Vervang 0-waarden door NaN zodat de imputer ze kan identificeren
# data.replace(0, np.nan, inplace=True)

# # Initialiseer de imputer om de gemiddelde waarde ('mean') te gebruiken
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# # Pas de imputer toe op de data
# # Dit vervangt elke NaN (oorspronkelijk 0) door het gemiddelde van de betreffende feature
# data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# # Controleer de eerste paar rijen van de aangepaste data
# print(data_imputed.head())




Aantal 0's per feature:
 label                                                       0
PREDICT_original_sf_compactness_avg_2.5D                    0
PREDICT_original_sf_compactness_std_2.5D                    0
PREDICT_original_sf_rad_dist_avg_2.5D                       0
PREDICT_original_sf_rad_dist_std_2.5D                       0
                                                         ... 
PREDICT_original_phasef_phasesym_peak_position_WL3_N5     115
PREDICT_original_phasef_phasesym_range_WL3_N5               0
PREDICT_original_phasef_phasesym_energy_WL3_N5              0
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5     12
PREDICT_original_phasef_phasesym_entropy_WL3_N5             0
Length: 494, dtype: int64


In [106]:


def load_and_split_data(csv_path, target_col, test_size=0.2, random_state=42):
    """
    Load dataset from a CSV file and split into training and testing sets.
    
    Parameters:
    - csv_path: str, path to the CSV file.
    - target_col: str, name of the target variable column.
    - test_size: float, proportion of the dataset to include in the test split (default is 0.2).
    - random_state: int, controls the shuffling applied to the data before applying the split (default is 42).
    
    Returns:
    - X_train: DataFrame, training set features.
    - X_test: DataFrame, testing set features.
    - y_train: Series, training set target.
    - y_test: Series, testing set target.
    """
    
    # Load the dataset
    df = pd.read_csv(csv_path)
    
    # Separate features and target variable
    x = df.drop(['label', 'ID'], axis=1)
    y = df[target_col]
    
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)
    
    return x_train, x_test, y_train, y_test

csv_path = '/Users/anneloormemelink/Documents/Machine learning/TM10007_ML_Project_G17/worclipo/Lipo_radiomicFeatures.csv'  # Adjust this path to your CSV file's location
target_col = 'label'  # Replace 'Target' with the actual name of your target column
test_size = 0.2  # 20% of the data will be used for testing

# Call the function
x_train, x_test, y_train, y_test = load_and_split_data(csv_path, target_col, test_size)

# Optionally, print the sizes of the splits to verify
print("Training set size:", x_train.shape)
print("Testing set size:", x_test.shape)


Training set size: (92, 493)
Testing set size: (23, 493)


In [107]:
# Standard scaler from sklearn, fitted on train data
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(x_train)

# Applying scaler to train and test set
X_train_scaled = scaler.transform(x_train)
X_test_scaled = scaler.transform(x_test)